In [1]:
#Import bibliotek
import psycopg2
from psycopg2 import connect

#Połączenie z bazą danych
username = 'nick'
password = 'haslo'
host = 'localhost'
database = 'airlines'
port = port

con = psycopg2.connect(
    dbname=database,
    user=username,
    password=password,
    host=host,
    port=port
)
cursor = con.cursor()

# Sprawdź, czy połączenie działa
cur = con.cursor()
cur.execute('SELECT version();')
print(cur.fetchone()[0])
cur.close()



#Wczytanie pliku reporting.sql
with open('sciezka do pliku', 'r') as reporting_file:
    reporting_sql = reporting_file.read()

#Rozdzielenie zawartości pliku
queries = reporting_sql.split(';')
queries = [query.strip() + ';' for query in queries if query.strip()]
for query in queries:
    print(query)
    
    

#Wykonanie kwerend, aby zainicjować strukturę bazy danych

# Utworzenie kursora
cur = con.cursor()
# Wykonanie kwerendy SQL
cur.execute("CREATE SCHEMA IF NOT EXISTS reporting")
# Potwierdzenie operacji i zakończenie połączenia
con.commit()

# Definicja widoku reporting.flight
reporting_flight = """
CREATE OR REPLACE VIEW reporting.flight AS
SELECT
    *,
    CASE
        WHEN dep_delay_new > 0 THEN 1
        ELSE 0
    END AS is_delayed
FROM
    public.flight
WHERE
    cancelled = 0
"""

# Definicja widoku reporting.top_reliability_roads
reporting_top_reliability_roads = """
CREATE OR REPLACE VIEW reporting.top_reliability_roads AS
SELECT
    subq.origin_airport_id,
    a.name AS origin_airport_name,
    subq.dest_airport_id,
    b.name AS dest_airport_name,
    subq.year,
    subq.cnt,
    subq.reliability,
    DENSE_RANK() OVER (PARTITION BY subq.year ORDER BY subq.reliability ASC) AS nb
FROM
    (SELECT
        f.origin_airport_id,
        f.dest_airport_id,
        f.year,
        COUNT(*) AS cnt,
        100.0 * SUM(CASE WHEN f.dep_delay_new > 0 THEN 1 ELSE 0 END) / COUNT(*) AS reliability
    FROM
        public.flight AS f
    GROUP BY
        f.origin_airport_id,
        f.dest_airport_id,
        f.year
    HAVING
        COUNT(*) > 10000) AS subq
JOIN
    airport_list AS a ON subq.origin_airport_id = a.origin_airport_id
JOIN
    airport_list AS b ON subq.dest_airport_id = b.origin_airport_id
"""

# Definicja widoku reporting.year_to_year_comparision
reporting_year_to_year_comparision = """
CREATE OR REPLACE VIEW reporting.year_to_year_comparision AS
SELECT
    f.year,
    f.month,
    COUNT(*) AS flights_amount,
    100.0 * SUM(CASE WHEN f.dep_delay_new > 0 THEN 1 ELSE 0 END) / COUNT(*) AS reliability
FROM
    public.flight AS f
GROUP BY
    f.year,
    f.month
"""
# Definicja widoku reporting.day_to_day_comparision
reporting_day_to_day_comparision = """
CREATE OR REPLACE VIEW reporting.day_to_day_comparision AS
SELECT
    f.year,
    f.day_of_week,
    COUNT(*) AS flights_amount
FROM
    public.flight AS f
GROUP BY
    f.year,
    f.day_of_week
"""

# Definicja widoku reporting.day_by_day_reliability
reporting_day_by_day_reliability = """
CREATE OR REPLACE VIEW reporting.day_by_day_reliability AS
SELECT
    TO_DATE(CONCAT(f.year, '-', LPAD(f.month::text, 2, '0'), '-', LPAD(f.day_of_month::text, 2, '0')), 'YYYY-MM-DD') AS date,
    100.0 * SUM(CASE WHEN f.dep_delay_new > 0 THEN 1 ELSE 0 END) / COUNT(*) AS reliability
FROM
    public.flight AS f
GROUP BY
    f.year,
    f.month,
    f.day_of_month;
"""

#Zatwierdzenie wszystkich operacji na bazie, czyli stworzenie widoków
cur = con.cursor()



# Wykonanie zapytań SQL
queries = [
    reporting_flight,
    reporting_top_reliability_roads,
    reporting_year_to_year_comparision,
    reporting_day_to_day_comparision,
    reporting_day_by_day_reliability
]

for query in queries:
    cur.execute(query)

# Zatwierdzenie zmian i zamknięcie połączenia
con.commit()


#Sprawdzenie

def check_if_table_exists(table_name):
    msg = f"I check if the table exists {table_name}"
    print(msg)

    query = f"select 1 from {table_name}"
    # jeżeli tabela nie istnieje, ten krok zwróci wyjątek
    cursor.execute(query)
    print('OK!')

views_to_test = [
    'reporting.flight',
    'reporting.top_reliability_roads',
    'reporting.year_to_year_comparision',
    'reporting.day_to_day_comparision',
    'reporting.day_by_day_reliability'
]

for view in views_to_test:
    check_if_table_exists(view)

con.close()